In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
from langchain_cohere import ChatCohere

llm = ChatCohere()

In [5]:
llm.invoke("how can langsmith help with testing?")

AIMessage(content="Langsmith can help with testing and quality assurance in several ways:\n\n1. **Language Testing**: Langsmith can generate language-specific test cases and scenarios to ensure that your software or product works correctly in different languages. This includes testing for proper translation, localization, and cultural adaptability.\n\n2. **Conversation Testing**: Langsmith can be used to create conversational test scripts and scenarios, especially for chatbots and voice assistants. You can use it to generate a variety of user inputs, questions, and commands to test the system's understanding and response accuracy.\n\n3. **Content Testing**: The tool is useful for generating content for testing purposes, such as user instructions, error messages, or help documentation. By generating content in different styles and tones, you can ensure that your content is clear, consistent, and effective across different use cases.\n\n4. **Edge Case Exploration**: Langsmith can help id

In [6]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world class technical documentation writer."),
    ("user", "{input}")
])

In [7]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [8]:
chain = prompt | llm | output_parser

In [9]:
chain.invoke({"input": "how can langsmith help with testing?"})

"Langsmith is a powerful tool that can greatly assist with testing and quality assurance processes. Here's how Langsmith can help:\n\n**1. Generate Test Data:** Langsmith can be used to generate a variety of realistic and contextually appropriate test data. You can create test cases, scenarios, and user inputs that mimic real-world usage, helping you identify and address potential issues before your product or service goes live.\n\n**2. Test Input Validation:** With its advanced language understanding capabilities, Langsmith can help test input validation and error handling. By generating a diverse range of inputs, including common variations, edge cases, and potential errors, you can ensure your system can handle different scenarios and provide appropriate responses.\n\n**3. Localization Testing:** If your product or service is targeted at a global audience, Langsmith's multilingual capabilities become especially useful. You can generate test content in multiple languages, helping you

In [10]:
from langchain_cohere.embeddings import CohereEmbeddings
embeddings = CohereEmbeddings()

In [11]:
from langchain_community.document_loaders import PyPDFLoader

In [27]:
from PyPDF2 import PdfReader
pdf_docs="documents\Budget_Speech.pdf"
pdf_reader= PdfReader(pdf_docs)

In [28]:
text=""
for page in pdf_reader.pages:
    text+= page.extract_text()

In [30]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=1000)
chunks = text_splitter.split_text(text)

In [37]:
from langchain_community.vectorstores import FAISS
vector_store = FAISS.from_texts(chunks, embedding=embeddings)
vector_store.save_local("faiss_index")

In [34]:
from langchain.chains import create_retrieval_chain

retriever = vector_store.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [36]:
response = retrieval_chain.invoke({"input": "tell me about amrit kaal from budget 2024."})
print(response["answer"])

# LangSmith offers several features that can help with testing:...

In the 2023-24 Budget Speech by the Finance Minister of India, Nirmala Sitharaman, the term "Amrit Kaal" refers to a period of growth, development, and transformation that the country aims to achieve as it moves towards its 100th year of independence in 2047. The vision for Amrit Kaal focuses on creating a technology-driven and knowledge-based economy with strong public finances and a robust financial sector.

To achieve this vision, the budget outlines seven key priorities, including inclusive development, reaching the last mile, infrastructure and investment, unleashing the potential, green growth, youth power, and strengthening the financial sector. The government aims to facilitate ample opportunities for citizens, especially the youth, to fulfill their aspirations, provide a strong impetus to growth and job creation, and enhance macroeconomic stability.

During Amrit Kaal, the government plans to empower women economically through the Deendayal Antyodaya Yojana National Rural Live

In [26]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)